Ce document contient les étapes de traitement des données du guide précarité du Codelico (Conseil Départemental du Bas-Rhin, 67) 

### Données originales

Les données originales du Codelico ont été téléchargées du site du Conseil Départemental.

Ces données sont des données publiques et tombent donc sous la clause Open Data de la Loi pour une République Numérique.

Ces données sont répertoriées sur la page https://welcome-guide.eu/guide-precarite-codelico/index.html

### Extraction des données 

Les données originales sont au format PDF. Leur extraction de manière automatique n'est pas aisée.

Ces données ont donc été extraites à la main, via environ 8 heures de travail divisé entre deux personnes (5/3).

Les données ont été entrées dans un tableur collaboratif (hébergé par l'association Framasoft) accessible à l'adresse https://lite.framacalc.org/OKiS1cuixs-codelico

La version finale de ce tableur collaboratif (25 juin 2019) est stockée dans ici : https://welcome-guide.eu/donnees-welcome-guide/OKiS1cuixs-codelico.csv

Cette version pourra être mise à jour avec les ajouts de certaines métadonnées (comme l'affichage dans le flyer ou dans la version travailleurs sociaux)

#### Téléchargement de la dernière version à jour des données extraites

In [ ]:
!wget https://lite.framacalc.org/OKiS1cuixs-codelico.csv -O ../donnees-welcome-guide/OKiS1cuixs-codelico.csv

### Traitement des données

Les données entrées dans le tableur collaboratif contiennent quelques lignes qui n'ont servi qu'à l'étape d'extraction des données.

#### Organisations dupliquées

De nombreuses organisations proposent de plusieurs services, et cela va demander de traiter les données de manière spécifique.

Pour pouvoir travailler sur les données des organisations qui ne proposent qu'un seul service, le plus simple est de les filtrer.

In [ ]:
!cat ../donnees-welcome-guide/OKiS1cuixs-codelico.csv |\
                    awk -F ',' '{print $1}' |\
                    sed 's_^" *__;s_ *"$__' |\
                    sort | uniq -d > ../donnees-welcome-guide/organisations-dupliquees.txt

In [ ]:
import pandas

In [ ]:
organisations_dupliquees = pandas.read_csv("../donnees-welcome-guide/organisations-dupliquees.txt")

In [ ]:
organisations_dupliquees.head(3)

### Nettoyage des noms des entités

Certains noms d'organisation sont présents avec des variations (présence d'espaces, présence ou absence d'accents, etc).

Cette étape vise à leur harmonisation

In [ ]:
tableau_extrait_codelico_no_header = pandas.read_csv("../donnees-welcome-guide/OKiS1cuixs-codelico.csv", skiprows=1)

In [ ]:
tableau_extrait_codelico_no_index = tableau_extrait_codelico_no_header.rename(columns=tableau_extrait_codelico_no_header.loc[1]).drop([0,1])

Certains noms commencent par une espace

In [ ]:
regexp_espace_precedant_nom = r"^ +"
masque_espace_precedant_nom = tableau_extrait_codelico_no_index.Nom.str.match(regexp_espace_precedant_nom, na=False)
tableau_extrait_codelico_no_index[masque_espace_precedant_nom]

In [ ]:
tableau_extrait_codelico_no_index.Nom = tableau_extrait_codelico_no_index.Nom.str.replace(regexp_espace_precedant_nom, "")

In [ ]:
tableau_extrait_codelico_no_index.head(4)

In [ ]:
masque_espace_precedant_nom = tableau_extrait_codelico_no_index.Nom.str.match(regexp_espace_precedant_nom, na=False)
tableau_extrait_codelico_no_index[masque_espace_precedant_nom]

Il n'y a pas d'organisation avec des espaces à la fin du nom.

In [ ]:
masque_espace_suivant_nom = tableau_extrait_codelico_no_index.Nom.str.match(r" $", na=False)
tableau_extrait_codelico_no_index[masque_espace_suivant_nom]

In [ ]:
import re

### Identification des noms d'organisation répétés

In [ ]:
noms_orgas = []

for name in tableau_extrait_codelico_no_index.Nom.sort_values():
    noms_orgas.append( re.sub(r" +", " ", name))

In [ ]:
for item in noms_orgas:
    print(item)

nom_orga_problematique = [
    "ABBA", 
    "ALT - Association de lutte contre la toxicomanie ",
    
]

In [ ]:
masque_orga_nom_problematique = tableau_extrait_codelico_no_index.Nom.sort_values().str.match("ALT")
tableau_extrait_codelico_no_index[masque_orga_nom_problematique].Nom

In [ ]:
for nom in tableau_extrait_codelico_no_index[masque_orga_nom_problematique].Nom:
    print(nom)

### Variations des noms d'organisations

Nous allons avoir besoin d'un dictionnaire pour associer à une organisation les variations sur son nom


In [ ]:
variations_nom_organisations = {}

Un travail est à faire à la main, de choisir un nom d'organisation principal.

In [ ]:
nom_organisation_principal = ["ALT - Association de lutte contre la toxicomanie  CSAPA - Centre d'accueil et de soins",
                             
                             
                             
                             ]

In [ ]:
tableaux_alphabetique = tableau_extrait_codelico_no_index.sort_values("Nom")

In [ ]:
masque_duplicated_orga = tableaux_alphabetique[["Nom", "adresse", "Ville"]].duplicated()
organisations_dupliquees = tableaux_alphabetique[masque_duplicated_orga].Nom

In [ ]:
tableau_extrait_codelico_no_index.columns

In [ ]:
for organisation in organisations_dupliquees:
    masque_organisation_dupliquee = tableau_extrait_codelico_no_index.Nom == organisation
    print("--------------------")
    print(tableau_extrait_codelico_no_index[masque_organisation_dupliquee]["Texte complémentaire Codelico"].values)

In [ ]:
organisations_dupliquees = tableau_extrait_codelico_no_index[tableau_extrait_codelico_no_index.Nom.duplicated()]

In [ ]:
len(organisations_dupliquees)

In [ ]:
organisations_dupliquees.Nom.sort_values

In [ ]:
a

### Set index

In [ ]:
tableau_extrait_codelico = tableau_extrait_codelico_no_index.set_index(["Nom"])

## tag:flyer



## Toujours à faire

- Ces données ne sont toutefois pas disponibles sur un portail Open Data d'une administration